In [16]:
import subprocess
import os
import ee
import sankee
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
import ipyleaflet
from ipyleaflet import WidgetControl
from ipywidgets import Layout, Button, Box, VBox
Map = geemap.Map(center=[29.1071 ,-8.8280 ], zoom=5.45, add_google_map=True, sandbox_path='./app')
Map.add_basemap('Esri Standard')


# Add Earth Engine data

Map

Map(center=[29.1071, -8.828], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [12]:
output_widget = widgets.Output(layout={'border': '2px solid #bfc2c7', 'height':'600px'})
output_control = WidgetControl(widget=output_widget)
output_widget.clear_output()


    

In [15]:
Map.add_legend(builtin_legend='NLCD',layer_name='lg', height='300px', legend_title= 'légende de la classification')

In [ ]:
polygon1=ee.Geometry.Polygon(
[[[-121.864019, 37.623243],
   [-121.864019, 37.755232],
   [-121.672733, 37.755232],
   [-121.672733, 37.623243],
   [-121.864019, 37.623243]]])
polygon2=ee.Geometry.Polygon(
[[[-122.526967, 37.959641],
   [-122.526967, 38.027922],
   [-122.437567, 38.027922],
   [-122.437567, 37.959641],
   [-122.526967, 37.959641]]])
polygon3=ee.Geometry.Polygon(
[[[-115.136336, 36.189253],
   [-115.136336, 36.340647],
   [-114.963528, 36.340647],
   [-114.963528, 36.189253],
   [-115.136336, 36.189253]]])

polygon4=ee.Geometry.Polygon(
[[[-106.426226, 31.863013],
   [-106.426226, 31.907535],
   [-106.351309, 31.907535],
   [-106.351309, 31.863013],
   [-106.426226, 31.863013]]])
labels={
            1: "No data",
            11: "Open water",
            12: "Perennial ice/snow",
            21: "Developed, open space",
            22: "Developed, low intensity",
            23: "Developed, medium intensity",
            24: "Developed, high intensity",
            31: "Barren land (rock/sand/clay)",
            41: "Deciduous forest",
            42: "Evergreen forest",
            43: "Mixed forest",
            51: "Dwarf scrub",
            52: "Shrub/scrub",
            71: "Grassland/herbaceous",
            72: "Sedge/herbaceous",
            73: "Lichens",
            74: "Moss",
            81: "Pasture/hay",
            82: "Cultivated crops",
            90: "Woody wetlands",
            95: "Emergent herbaceous wetlands",
        }
palette={
            1: "#000000",
            11: "#466b9f",
            12: "#d1def8",
            21: "#dec5c5",
            22: "#d99282",
            23: "#eb0000",
            24: "#ab0000",
            31: "#b3ac9f",
            41: "#68ab5f",
            42: "#1c5f2c",
            43: "#b5c58f",
            51: "#af963c",
            52: "#ccb879",
            71: "#dfdfc2",
            72: "#d1d182",
            73: "#a3cc51",
            74: "#82ba9e",
            81: "#dcd939",
            82: "#ab6c28",
            90: "#b8d9eb",
            95: "#6c9fb8",
        }
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)
style = {'description_width': 'initial'}
classification_type = widgets.Dropdown(
    description='method:',
    options=['SmileCart', 'RandomForest'],
    value='RandomForest',
    style=style,
    layout=form_item_layout
)
year_widget001 = widgets.IntSlider(min=1984, max=2020, value=2017, description='Selected year1:', width=400, style=style,  justify_content='space-between')
classif = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='classify',
    style=style
)
year_widget002 = widgets.IntSlider(min=1984, max=2020, value=2019, description='Selected year2:', width=400, style=style,  justify_content='space-between')


def renamels(img):
        return img.select(
            ['Blue','Green','Red','NIR', 'SWIR1','SWIR2','pixel_qa'],
            ['B2', 'B3', 'B4', 'B5','B6', 'B7', 'pixel_qa'],
        )



# classify function
def classif_clicked(b):
    visualization = {
                    'bands': ['classification']
                       }
    output_widget.clear_output()
    year001=year_widget001.value
    year002=year_widget002.value
    roi=Map.user_roi
    images2 = geemap.landsat_timeseries(roi=roi, start_year=year001, end_year=year002, start_date='01-01', end_date='12-31', apply_fmask=True)
    selected_image = ee.Image(images2.toList(images2.size()).get(0))
    selected_image2 = ee.Image(images2.toList(images2.size()).get(year002-year001))
    images3 = geemap.landsat_timeseries(roi=polygon1, start_year=2016, end_year=2017, start_date='01-01', end_date='12-31', apply_fmask=True)
    images4 = geemap.landsat_timeseries(roi=polygon2, start_year=2016, end_year=2017, start_date='01-01', end_date='12-31', apply_fmask=True)
    images5 = geemap.landsat_timeseries(roi=polygon3, start_year=2016, end_year=2017, start_date='01-01', end_date='12-31', apply_fmask=True)
    images6 = geemap.landsat_timeseries(roi=polygon4, start_year=2016, end_year=2017, start_date='01-01', end_date='12-31', apply_fmask=True)
    alpha1 = ee.Image(images3.toList(images3.size()).get(0))
    alpha2 = ee.Image(images4.toList(images4.size()).get(0))
    alpha3 = ee.Image(images5.toList(images4.size()).get(0))
    alpha4 = ee.Image(images6.toList(images5.size()).get(0))

    trainarea1=renamels(alpha1)
    trainarea2=renamels(alpha2)
    trainarea3=renamels(alpha3)
    trainarea4=renamels(alpha4)
    
    #image=renamehhh(selected_image)
    image=renamels(selected_image)
    image_two=renamels(selected_image2)
    #point = ee.Geometry.Point([-2.713931, 38.581355])
#     point = ee.Geometry.Point([48.4583, 32.6310])
#     point2 = ee.Geometry.Point([51.4069, 35.7006])
#     point3 = ee.Geometry.Point([49.6715, 34.0937])

#-122.4439, 37.7538
    #bands = ['B2', 'B3', 'B4','B8','B11','B12']
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# point = ee.Geometry.Point([-87.7719, 41.8799])

    
   
    #nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image0.geometry())
    nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
    #Map.addLayer(nlcd,visualization,'grtruth')
    #Map.addLayer(image0,visParams,'image0')
    points1 = nlcd.sample(**{
             'region': polygon1,
             'scale': 30,
             'numPixels': 3000,
             'seed': 0,
             'geometries': True  # Set this to False to ignore geometries
             })
    points2 = nlcd.sample(**{
             'region': polygon2,
             'scale': 30,
             'numPixels': 3000,
             'seed': 0,
             'geometries': True  # Set this to False to ignore geometries
             })
    points3 = nlcd.sample(**{
             'region': polygon3,
             'scale': 30,
             'numPixels': 3000,
             'seed': 0,
             'geometries': True  # Set this to False to ignore geometries
             })
    points4 = nlcd.sample(**{
             'region': polygon4,
             'scale': 30,
             'numPixels': 3000,
             'seed': 0,
             'geometries': True  # Set this to False to ignore geometries
             })

    label = 'landcover'
    #label = 'landcover'
    #Map.addLayer(points1,{},'pts',False)
    #Map.addLayer(points2,{},'pts2',False)
    #Map.addLayer(points3,{},'pts3',False)
    # Overlay the points on the imagery to get training.
    training1 = trainarea1.select(bands).sampleRegions(**{
              'collection': points1,
              'properties': [label],
              'scale': 30
               })
    training2 = trainarea2.select(bands).sampleRegions(**{
              'collection': points2,
              'properties': [label],
              'scale': 30
               })
    training3 = trainarea3.select(bands).sampleRegions(**{
              'collection': points3,
              'properties': [label],
              'scale': 30
               })
    training4 = trainarea4.select(bands).sampleRegions(**{
              'collection': points4,
              'properties': [label],
              'scale': 30
               })


    
    training1_1=training1.merge(training2)
    training1_2=training1_1.merge(training3)
    training=training1_2.merge(training4)
    if classification_type.value == 'SmileCart':
        trained = ee.Classifier.smileCart().train(training, label, bands)
    elif classification_type.value == 'RandomForest':
        trained= ee.Classifier.smileRandomForest(10).train(training, label, bands) 

    result = image.select(bands).classify(trained)
    result2=image_two.select(bands).classify(trained)
    class_values = nlcd.get('landcover_class_values').getInfo()
    class_palette = nlcd.get('landcover_class_palette').getInfo()
    landcover = result.set('classification_class_values', class_values)
    landcover2 = result2.set('classification_class_values', class_values)
    landcover = landcover.set('classification_class_palette', class_palette)
    landcover2 = landcover2.set('classification_class_palette', class_palette)
    y1=str(year001)
    y2=str(year002)
    Map.addLayer(landcover,{},y1,False)
    Map.addLayer(landcover2,{}, y2)
    #Map.addLayer(points0, {}, 'pts',False)
    #output_widgetx.clear_output()
    #with output_widgetx:
        #train_accuracy = trained.confusionMatrix()
       # print('train accuracy :',train_accuracy.accuracy().getInfo())
        #print('kappa :', train_accuracy.kappa().getInfo())
       # print('confusion matrix :',np.asmatrix(train_accuracy.getInfo()))
    
    band ="classification"
    img_list = [landcover, landcover2]
    label_list = ["year1", "year2"]
    with output_widget:
            output_widget.clear_output()
            logo11 = widgets.HTML(value='<center><div style="padding-top:160px"><img src="https://1.bp.blogspot.com/-ra3PLTIEFcc/VtSzvxuWaBI/AAAAAAAAA1k/YyxLnMx1jjM/s320/sea_globe_50fr_150ms.gif" width="100" height="100"><br/><h3>processing ...</h3></center></div>')
            display(logo11)
            Map.default_style = {'cursor': 'wait'}
            try:
                plot = sankee.sankify(
                      img_list,
                      region=roi,
                      label_list=label_list,
                      band=band,
                      labels=labels,
                      palette=palette,
                      scale=30
                      )
                output_widget.clear_output()
                display(plot)    
            #print(landcover2.getInfo())
            except Exception as e:
                      print(e)
                      print('An error occurred during computation.') 





In [ ]:
classif.on_click(classif_clicked) 

In [ ]:


layer_names = ['Landsat ' + str(year) for year in range(1984, 2020)]
style = {'description_width': 'initial'}

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)


band_combo = widgets.Dropdown(
    description='Band combo:',
    options=['Red/Green/Blue', 'NIR/Red/Green',  'SWIR2/SWIR1/NIR', 'NIR/SWIR1/Red','SWIR2/NIR/Red', 
             'SWIR2/SWIR1/Red', 'SWIR1/NIR/Blue', 'NIR/SWIR1/Blue', 'SWIR2/NIR/Green', 'SWIR1/NIR/Red'],
    value='NIR/Red/Green',
    style=style,
    layout=form_item_layout
)

nd_options = ['Vegetation Index (NDVI)', 
              'Water Index (NDWI)',
              'Modified Water Index (MNDWI)',
              'N. Built-up Index (NDBI)',
              'DBSI',
              'Customized',
              ]
nd_indices = widgets.Dropdown(options=nd_options, value='Modified Water Index (MNDWI)', description='Index:', style=style,  justify_content='space-between')

year_widget1 = widgets.IntSlider(min=1984, max=2020, value=1992, description='Selected year1:', width=400, style=style,  justify_content='space-between')
year_widget2 = widgets.IntSlider(min=1984, max=2020, value=2010, description='Selected year2:', width=400, style=style,  justify_content='space-between')

first_band = widgets.Dropdown(
    description='1st band:',
    options=['Blue', 'Green','Red','NIR', 'SWIR1', 'SWIR2'],
    value='Green',
    style=style
)

second_band = widgets.Dropdown(
    description='2nd band:',
    options=['Blue', 'Green','Red','NIR', 'SWIR1', 'SWIR2'],
    value='SWIR1',
    style=style
)
def nd_index_change(change):
    if nd_indices.value == 'Vegetation Index (NDVI)':
        first_band.value = 'NIR'
        second_band.value = 'Red'
    elif nd_indices.value == 'Water Index (NDWI)':
        first_band.value = 'NIR'
        second_band.value = 'SWIR1'   
    elif nd_indices.value == 'Modified Water Index (MNDWI)':
        first_band.value = 'Green'
        second_band.value = 'SWIR1'   
    elif nd_indices.value == 'N. Built-up Index (NDBI)':
        first_band.value = 'SWIR1'
        second_band.value = 'NIR'        
       
    else:
        first_band.value = None
        second_band.value = None

        
nd_indices.observe(nd_index_change, names='value')


nd_threshold = widgets.FloatSlider(
    value=0.4,
    min=-1,
    max=1,
    step=0.01,
    description='Threshold:',
    orientation='horizontal',
    style=style
)

nd_color = widgets.ColorPicker(
    concise=False,
    description='Color:',
    value='blue',
    style=style
)
year_widget = widgets.IntSlider(min=2017, max=2020, value=2017, description='Selected year:', width=400, style=style)

submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)
Detect_Buildings = widgets.Button(
    description='classify',
    button_style='primary',
    tooltip='Click me',
    style=style
)
text1= widgets.HTML(
    value=" <b>NLCD based classification</b>",
    placeholder='',
    description='',
)
text2= widgets.HTML(
    value=" <b>different indices with landsat timeseries</b>",
    placeholder='',
    description='',
)

output_widget2 = widgets.Output(layout={'border': '5px solid white'})

    
full_widget1 = widgets.VBox([text1, year_widget001,year_widget002,classification_type, classif])
full_widget = widgets.VBox([
    text2, 
    widgets.HBox([nd_indices]), 
    widgets.HBox([year_widget1,first_band]),
    widgets.HBox([year_widget2,second_band]),
    widgets.HBox([nd_color,nd_threshold]),
    submit,
 
])


cadre1=widgets.Output(layout={'border': '2px solid #bfc2c7'})
with cadre1:
    display(full_widget)
cadre2=widgets.Output(layout={'border': '2px solid #bfc2c7'})
with cadre2:
    display(full_widget1)

h= widgets.HBox([cadre1, cadre2])
h

In [ ]:
 output_widget

In [ ]:
def submit_clicked(b):
    output_widget.clear_output()
    with output_widget:
        logo11 = widgets.HTML(
        value='<center><div style="padding-top:160px"><img src="https://1.bp.blogspot.com/-ra3PLTIEFcc/VtSzvxuWaBI/AAAAAAAAA1k/YyxLnMx1jjM/s320/sea_globe_50fr_150ms.gif" width="100" height="100"><br/><h3>processing ...</h3></center></div>'
            )
        display(logo11)
        Map.default_style = {'cursor': 'wait'}
        try:
                vis_params = {
                'min': 0,
                'max': 4000,
                'gamma': [1, 1, 1],
                'bands': ['NIR', 'Red', 'Green']}
                band1 = first_band.value
                band2 = second_band.value
                band111='SWIR1'
                band31='Green'
                band41='Red'
                band81='NIR'
                threshold = nd_threshold.value
                selected_year1 = year_widget1.value
                selected_year2 = year_widget2.value
                palette = nd_color.value
                roi = Map.user_roi
                layer_name = 'User drawn AOI'
                geom = roi
                images = geemap.landsat_timeseries(roi=roi, start_year=1984, end_year=2020, start_date='01-01', end_date='12-31')
                images1 = ee.Image(images.toList(images.size()).get(selected_year1 -1984))
                if nd_indices.value=='DBSI' :
                    nd_images = images.map(lambda img: img.normalizedDifference([band111, band31]).subtract(img.normalizedDifference([band81, band41])))
                    result_images = nd_images.map(lambda img: img.gt(threshold))    
                else:    
                    nd_images = images.map(lambda img: img.normalizedDifference([band1, band2]))
                    result_images = nd_images.map(lambda img: img.gt(threshold))
        
                 


            
    
    
    
    
                selected_image1 = ee.Image(images.toList(images.size()).get(selected_year1 - 1984))
                selected_image2 = ee.Image(images.toList(images.size()).get(selected_year2 - 1984))
    #Map.split_map(left_layer=selected_image1, right_layer=selected_image2)
                selected_result_image1 = ee.Image(result_images.toList(result_images.size()).get(selected_year1 - 1984)).selfMask()
                selected_result_image2 = ee.Image(result_images.toList(result_images.size()).get(selected_year2 - 1984)).selfMask()
    #Map.addLayer(selected_image1, vis_params, 'Landsat ' + str(selected_year1))
#                 Map.addLayer(selected_result_image1, {'palette': palette}, 'Result ' + str(selected_year1),False)
#                 Map.addLayer(selected_result_image2, {'palette': palette}, 'Result ' + str(selected_year2),False)
    #Map.addLayer(selected_image2, vis_params, 'Landsat ' + str(selected_year2))
    #Map.addLayer(selected_result_image1, {'palette': palette}, 'Result ' + str(selected_year2))
                left_layer = geemap.ee_tile_layer(selected_result_image1, {'palette': palette}, 'Landsat ' + str(selected_year1))
                right_layer = geemap.ee_tile_layer(selected_result_image2, {'palette': palette}, 'Landsat ' + str(selected_year2))

                Map.split_map(left_layer, right_layer)
            

    #landsat_ts = geemap.landsat_timeseries(roi=roi, start_year=1984, end_year=2019, start_date='01-01', end_date='12-31')
    #bands = band_combo.value.split('/')

    #Map.ts_inspector(left_ts=landsat_ts, right_ts=landsat_ts, left_names=layer_names, right_names=layer_names, left_vis=landsat_vis, right_vis=landsat_vis)
    #Map.centerObject(roi, zoom=8)
    
    
    
            
            
#                 def cal_area(img):
#                     pixel_area = img.multiply(ee.Image.pixelArea()).divide(1e4)
#                     img_area = pixel_area.reduceRegion(**{
#                         'geometry': roi,
#                         'reducer': ee.Reducer.sum(),
#                         'scale': 1000,
#                         'maxPixels': 1e12,
#                         'bestEffort': True
#                     })
#                     return img.set({'area': img_area})
               
#                 areas = result_images.map(cal_area)
#                 stats = areas.aggregate_array('area').getInfo()
#                 x = list(range(selected_year1, selected_year2+1))
#                 y = [item.get('nd') for item in stats]
#                 fig = plt.figure(1)
#                 fig.layout.height = '270px'
#                 plt.clear()
#                 plt.plot(x, y)
#                 plt.title('Temporal trend'+str(selected_year1)+'-'+str(selected_year2))
#                 plt.xlabel('Year')
#                 plt.ylabel('Area (ha)')
            
#                 output_widget.clear_output()            

#                 plt.show()

            

    
        except Exception as e:
            print(e)
            print('An error occurred during computation.')        

        Map.default_style = {'cursor': 'default'}
   
